# Exp to launch

In [1]:
import sys

sys.path.append("paper-ner-bench-das22/src/ocr")  # XXX: Change ME

In [3]:
from pathlib import Path

import pandas as pd
from evaluate import load  # Hugging Face
from sklearn.model_selection import train_test_split

from src.controller import LengthController
from src.error_detector import ErrorDetector
from src.NMT_corrector import NMTCorrector
from src.pipeline import Pipeline

## Dataloading

In [3]:
from src.icdar_utils import get_df

root_icdar_path = Path("data/icdar_dataset")
train_path = (
    root_icdar_path / "ICDAR2019_POCR_competition_training_18M_without_Finnish/FR"
)  # XXX: Change ME
test_path = (
    root_icdar_path / "ICDAR2019_POCR_competition_evaluation_4M_without_Finnish/FR"
)  # XXX: Change ME

train_df = get_df(train_path, align_input=True, filter_aligned_error=True)
test_df = get_df(test_path, align_input=True, filter_aligned_error=True)

## Data prep

In [4]:
def chunkify(sample: str, *, chunk_size: int = 200):
    return [
        sample[start : start + chunk_size]
        for start in range(0, len(sample), chunk_size)
    ]


train_df["chunked_sample"] = train_df["Sample"].apply(chunkify)
train_df["chunked_gt"] = train_df["Ground Truth"].apply(chunkify)

test_df["chunked_sample"] = test_df["Sample"].apply(chunkify)
test_df["chunked_gt"] = test_df["Ground Truth"].apply(chunkify)

train_df

,Sample,Ground Truth,chunked_sample,chunked_gt
0,4 #» Clttf CRF-CITY LA ROCHELLE 33 RUE DE LA S...,@@@@@@@@@@@CRF-CITY LA ROCHELLE 33 RUE DE LA S...,[4 #» Clttf CRF-CITY LA ROCHELLE 33 RUE DE LA ...,[@@@@@@@@@@@CRF-CITY LA ROCHELLE 33 RUE DE LA ...
1,TSEO BISTRO! DE LA MER 4 PLACE DE LA CHAINE - ...,TSEO BISTROT DE LA MER 4 PLACE DE LA CHAINE – ...,[TSEO BISTRO! DE LA MER 4 PLACE DE LA CHAINE -...,[TSEO BISTROT DE LA MER 4 PLACE DE LA CHAINE –...
2,"<*» cittj CRF-C@HY LA ROCHELLE ^ 33 RUE DE ,A ...",@@@@ci@ty CRF-CITY LA ROCHELLE@@ 33 RUE DE LA ...,"[<*» cittj CRF-C@HY LA ROCHELLE ^ 33 RUE DE ,A...",[@@@@ci@ty CRF-CITY LA ROCHELLE@@ 33 RUE DE LA...
3,<#» City CRF-CITY LA ROCHELLE 33 RUE DE LA SCI...,@@@@city CRF-CITY LA ROCHELLE 33 RUE DE LA SCI...,[<#» City CRF-CITY LA ROCHELLE 33 RUE DE LA SC...,[@@@@city CRF-CITY LA ROCHELLE 33 RUE DE LA SC...
4,Ni mi: 7075 Sor ie:@@@@@@@@@@01@ 1@@@@@@7 @@1@...,N@um@: 2025 Ser@ie:01 12-10-2016 12:52:49 (V1)...,[Ni mi: 7075 Sor ie:@@@@@@@@@@01@ 1@@@@@@7 @@1...,[N@um@: 2025 Ser@ie:01 12-10-2016 12:52:49 (V1...
...,...,...,...,...
1878,"Philippes, etc. Savoir fai@@@s à @ouz@ presens...","Philippes, etc. Savoir faisons à touz, presens...","[Philippes, etc. Savoir fai@@@s à @ouz@ presen...","[Philippes, etc. Savoir faisons à touz, presen..."
1879,"Johannes, Dei gratia Francorum rex. Notum faci...","Johannes, Dei gracia Francorum rex. Notum faci...","[Johannes, Dei gratia Francorum rex. Notum fac...","[Johannes, Dei gracia Francorum rex. Notum fac..."
1880,"Loys, etc., au seneschal de Poictou ou à son l...","Loys, etc., au seneschal de Poictou ou à son l...","[Loys, etc., au seneschal de Poictou ou à son ...","[Loys, etc., au seneschal de Poictou ou à son ..."
1881,"Charles, etc. Savoir faisons à touz, presens e...","Charles, etc. Savoir faisons à touz, presens e...","[Charles, etc. Savoir faisons à touz, presens ...","[Charles, etc. Savoir faisons à touz, presens ..."


In [5]:
ALIGNMENT_CHAR = ["@", "#"]


def clean_alignment(sentence: str) -> str:
    for char in ALIGNMENT_CHAR:
        sentence = sentence.replace(char, "")

    return sentence


train_df = train_df.explode(["chunked_sample", "chunked_gt"])
test_df = test_df.explode(["chunked_sample", "chunked_gt"])

train_df["Clean Sample"] = train_df["chunked_sample"].apply(clean_alignment)
train_df["Clean Ground Truth"] = train_df["chunked_gt"].apply(clean_alignment)

test_df["Clean Sample"] = test_df["chunked_sample"].apply(clean_alignment)
test_df["Clean Ground Truth"] = test_df["chunked_gt"].apply(clean_alignment)

train_df

,Sample,Ground Truth,chunked_sample,chunked_gt,Clean Sample,Clean Ground Truth
0,4 #» Clttf CRF-CITY LA ROCHELLE 33 RUE DE LA S...,@@@@@@@@@@@CRF-CITY LA ROCHELLE 33 RUE DE LA S...,4 #» Clttf CRF-CITY LA ROCHELLE 33 RUE DE LA S...,@@@@@@@@@@@CRF-CITY LA ROCHELLE 33 RUE DE LA S...,4 » Clttf CRF-CITY LA ROCHELLE 33 RUE DE LA SC...,CRF-CITY LA ROCHELLE 33 RUE DE LA SCIERIE 1700...
0,4 #» Clttf CRF-CITY LA ROCHELLE 33 RUE DE LA S...,@@@@@@@@@@@CRF-CITY LA ROCHELLE 33 RUE DE LA S...,Monnaie -1.77s 0005 002 000255 25/03/2017 18:...,Monnaie -1.77€ 0005 002 000255 25/03/2017 18:...,Monnaie -1.77s 0005 002 000255 25/03/2017 18:...,Monnaie -1.77€ 0005 002 000255 25/03/2017 18:...
1,TSEO BISTRO! DE LA MER 4 PLACE DE LA CHAINE - ...,TSEO BISTROT DE LA MER 4 PLACE DE LA CHAINE – ...,TSEO BISTRO! DE LA MER 4 PLACE DE LA CHAINE - ...,TSEO BISTROT DE LA MER 4 PLACE DE LA CHAINE – ...,TSEO BISTRO! DE LA MER 4 PLACE DE LA CHAINE - ...,TSEO BISTROT DE LA MER 4 PLACE DE LA CHAINE – ...
1,TSEO BISTRO! DE LA MER 4 PLACE DE LA CHAINE - ...,TSEO BISTROT DE LA MER 4 PLACE DE LA CHAINE – ...,"ANE BIST Offert PERRIER 3,90 RICARD 3.80 PINEA...",ANE BIST Offert PERRIER 3.90 RICARD 3.80 PINEA...,"ANE BIST Offert PERRIER 3,90 RICARD 3.80 PINEA...",ANE BIST Offert PERRIER 3.90 RICARD 3.80 PINEA...
1,TSEO BISTRO! DE LA MER 4 PLACE DE LA CHAINE - ...,TSEO BISTROT DE LA MER 4 PLACE DE LA CHAINE – ...,E SIRET 537 924 755 000 11 LA ROCHELL TVA INTR...,E SIRET 537 924 755 000 11 LA ROCHELL TVA INTR...,E SIRET 537 924 755 000 11 LA ROCHELL TVA INTR...,E SIRET 537 924 755 000 11 LA ROCHELL TVA INTR...
...,...,...,...,...,...,...
1882,"Charles, etc. Savoir faisons à touz, presens e...","Charles, etc. Savoir faisons à touz, presens e...",emettre. et se fust transporté ou païs d@oieng...,"emettre, et se fust transporté ou païs d’Oreng...",emettre. et se fust transporté ou païs doienge...,"emettre, et se fust transporté ou païs d’Oreng..."
1882,"Charles, etc. Savoir faisons à touz, presens e...","Charles, etc. Savoir faisons à touz, presens e...",aïs de Poitou et environ en nostre royaume@ et...,"aïs de Poitou et environ en nostre royaume, et...",aïs de Poitou et environ en nostre royaume et ...,"aïs de Poitou et environ en nostre royaume, et..."
1882,"Charles, etc. Savoir faisons à touz, presens e...","Charles, etc. Savoir faisons à touz, presens e...",é moult grandement@ priz et detenuz en prison ...,"é moult grandement, priz et detenuz en prison ...",é moult grandement priz et detenuz en prison p...,"é moult grandement, priz et detenuz en prison ..."
1882,"Charles, etc. Savoir faisons à touz, presens e...","Charles, etc. Savoir faisons à touz, presens e...",estre poursui et molesté par noz genz et offic...,estre poursui et molesté par noz genz et offic...,estre poursui et molesté par noz genz et offic...,estre poursui et molesté par noz genz et offic...


In [6]:
# Re-index for compatibility with Hugging face
train_df["original idx"] = train_df.index
test_df["original idx"] = test_df.index

train_df = train_df.reset_index()
test_df = test_df.reset_index()

train_df

,index,Sample,Ground Truth,chunked_sample,chunked_gt,Clean Sample,Clean Ground Truth,original idx
0,0,4 #» Clttf CRF-CITY LA ROCHELLE 33 RUE DE LA S...,@@@@@@@@@@@CRF-CITY LA ROCHELLE 33 RUE DE LA S...,4 #» Clttf CRF-CITY LA ROCHELLE 33 RUE DE LA S...,@@@@@@@@@@@CRF-CITY LA ROCHELLE 33 RUE DE LA S...,4 » Clttf CRF-CITY LA ROCHELLE 33 RUE DE LA SC...,CRF-CITY LA ROCHELLE 33 RUE DE LA SCIERIE 1700...,0
1,0,4 #» Clttf CRF-CITY LA ROCHELLE 33 RUE DE LA S...,@@@@@@@@@@@CRF-CITY LA ROCHELLE 33 RUE DE LA S...,Monnaie -1.77s 0005 002 000255 25/03/2017 18:...,Monnaie -1.77€ 0005 002 000255 25/03/2017 18:...,Monnaie -1.77s 0005 002 000255 25/03/2017 18:...,Monnaie -1.77€ 0005 002 000255 25/03/2017 18:...,0
2,1,TSEO BISTRO! DE LA MER 4 PLACE DE LA CHAINE - ...,TSEO BISTROT DE LA MER 4 PLACE DE LA CHAINE – ...,TSEO BISTRO! DE LA MER 4 PLACE DE LA CHAINE - ...,TSEO BISTROT DE LA MER 4 PLACE DE LA CHAINE – ...,TSEO BISTRO! DE LA MER 4 PLACE DE LA CHAINE - ...,TSEO BISTROT DE LA MER 4 PLACE DE LA CHAINE – ...,1
3,1,TSEO BISTRO! DE LA MER 4 PLACE DE LA CHAINE - ...,TSEO BISTROT DE LA MER 4 PLACE DE LA CHAINE – ...,"ANE BIST Offert PERRIER 3,90 RICARD 3.80 PINEA...",ANE BIST Offert PERRIER 3.90 RICARD 3.80 PINEA...,"ANE BIST Offert PERRIER 3,90 RICARD 3.80 PINEA...",ANE BIST Offert PERRIER 3.90 RICARD 3.80 PINEA...,1
4,1,TSEO BISTRO! DE LA MER 4 PLACE DE LA CHAINE - ...,TSEO BISTROT DE LA MER 4 PLACE DE LA CHAINE – ...,E SIRET 537 924 755 000 11 LA ROCHELL TVA INTR...,E SIRET 537 924 755 000 11 LA ROCHELL TVA INTR...,E SIRET 537 924 755 000 11 LA ROCHELL TVA INTR...,E SIRET 537 924 755 000 11 LA ROCHELL TVA INTR...,1
...,...,...,...,...,...,...,...,...
15794,1882,"Charles, etc. Savoir faisons à touz, presens e...","Charles, etc. Savoir faisons à touz, presens e...",emettre. et se fust transporté ou païs d@oieng...,"emettre, et se fust transporté ou païs d’Oreng...",emettre. et se fust transporté ou païs doienge...,"emettre, et se fust transporté ou païs d’Oreng...",1882
15795,1882,"Charles, etc. Savoir faisons à touz, presens e...","Charles, etc. Savoir faisons à touz, presens e...",aïs de Poitou et environ en nostre royaume@ et...,"aïs de Poitou et environ en nostre royaume, et...",aïs de Poitou et environ en nostre royaume et ...,"aïs de Poitou et environ en nostre royaume, et...",1882
15796,1882,"Charles, etc. Savoir faisons à touz, presens e...","Charles, etc. Savoir faisons à touz, presens e...",é moult grandement@ priz et detenuz en prison ...,"é moult grandement, priz et detenuz en prison ...",é moult grandement priz et detenuz en prison p...,"é moult grandement, priz et detenuz en prison ...",1882
15797,1882,"Charles, etc. Savoir faisons à touz, presens e...","Charles, etc. Savoir faisons à touz, presens e...",estre poursui et molesté par noz genz et offic...,estre poursui et molesté par noz genz et offic...,estre poursui et molesté par noz genz et offic...,estre poursui et molesté par noz genz et offic...,1882


# Training pipeline

In [ ]:
pipeline = Pipeline(
    error_detector=ErrorDetector(),
    nmt_corrector=NMTCorrector(),
    controller=LengthController(),
)

In [ ]:
pipeline.fit(train_df["Clean Sample"], train_df["Clean Ground Truth"])

In [ ]:
pipeline.score(test_df["Clean Sample"], test_df["Clean Ground Truth"])

# Testing new Controller

In [2]:
from copy import copy

from src.controller import InsertDeletionController, compute_keep_decision

no_controller_pipeline = copy(pipeline)
no_controller_pipeline.controller = None

In [13]:
tmp_df = pd.DataFrame()
tmp_df["Sample"] = train_df["Clean Sample"]
tmp_df["Predictions"] = no_controller_pipeline.predict(train_df["Clean Sample"])
tmp_df["Ground Truth"] = train_df["Clean Ground Truth"]

X_train_controller = tmp_df.loc[:, ("Sample", "Predictions")]
X_train_controller = np.array(X_train_controller)

y_train_controller = tmp_df.apply(compute_keep_decision, axis=1)

In [ ]:
new_controller = InsertDeletionController()
new_controller.fit(X_train_controller, y_train_controller)

In [ ]:
new_pipeline = Pipeline(
    error_detector=pipeline.error_detector,
    nmt_corrector=pipeline.nmt_corrector,
    controller=new_controller,
)

new_pipeline.score(test_df["Clean Sample"], test_df["Clean Ground Truth"])